## BOOTSTRAPING 

**JULIA HERNÁNDEZ CÁRDENAS**

Utiliza los conceptos aprendidos en los laboratorios de regresión y clasificación para encontrar el error estándar de los coeficientes de una regresión (lineal/logística) simple para los datasets de “Advertising” y “Default”.

Utiliza bootstrap para simular 1000 remuestreos de esos datasets y calcula la media de los coeficientes obtenidos al aplicarle regresión a cada remuestreo. Calcula la desviación estándar.

Compara los resultados obtenidos con el método visto en los laboratorios contra los resultados obtenidos con bootstrap. ¿Por qué podría haber diferencias en los resultados?

Agrega regularización L2 a los modelos del dataset de Advertising (optimiza el hiperparámetro). Utiliza ese valor del hiperparámetro para repetir el experimento de los 1000 remuestreos. Calcula la desviación estándar de los coeficientes obtenidos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import random
from scipy.stats import t, norm
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error, log_loss
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold

## ADVERTISING

In [2]:
ad = pd.read_csv("Advertising.csv")
ad.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [3]:
X_todo = ad[['TV', 'radio', 'newspaper']]
X_todo = sm.add_constant(X_todo)
y_todo = ad['sales']

ols = sm.OLS(y_todo, X_todo)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 20 Nov 2025   Prob (F-statistic):           1.58e-96
Time:                        17:29:50   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
radio          0.1885      0.009     21.893      0.000       0.172       0.206
newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
coef_ols = results.params
se_ols = results.bse

In [5]:
B = 1000
coef_boot = []

n = len(ad)

for b in range(B):
    i = np.random.choice(n, n, replace=True)
    X_b = X_todo.iloc[i]
    y_b = y_todo.iloc[i]

    modelo_b = sm.OLS(y_b, X_b).fit()
    
    coef_boot.append(modelo_b.params.values)

coef_boot = np.array(coef_boot)

coef_boot_media = coef_boot.mean(axis=0)

coef_boot_std = coef_boot.std(axis=0)

In [6]:
comparacion = pd.DataFrame({
    "Coef OLS": coef_ols.values,
    "SE OLS": se_ols.values,
    "Media Bootstrap": coef_boot_media,
    "Std Bootstrap": coef_boot_std
}, index=X_todo.columns)

comparacion

,Coef OLS,SE OLS,Media Bootstrap,Std Bootstrap
const,2.938889,0.311908,2.940589,0.317533
TV,0.045765,0.001395,0.045659,0.001908
radio,0.188530,0.008611,0.188948,0.010490
newspaper,-0.001037,0.005871,-0.000856,0.006089


Podría existir una diferencia entre los resultados del análisis con y sin bootstrap ya que, con el modelo inicial de OLS, se asume normalidad y relaciones lineales entre las variables, mientras que con bootstrap se refleja más variabilidad entre los resultados. 
Tomando newspaper como ejemplo, el modelo con bootstrap muestra una desviación estándar mayor porque esta variable es un poco más débil y tiene mayor colinealidad con las otras. 

In [7]:
alphas = np.logspace(-4, 4, 200)
kfolds = KFold(n_splits=5, shuffle=True, random_state=123)

mejor_alpha = None
mejor_mse = np.inf

for a in alphas:
    mses = []
    for train, test in kfolds.split(X_todo):
        X_train, X_test = X_todo.iloc[train], X_todo.iloc[test]
        y_train, y_test = y_todo.iloc[train], y_todo.iloc[test]

        ridge = Ridge(alpha=a, fit_intercept=False)
        ridge.fit(X_train, y_train)

        preds = ridge.predict(X_test)
        mses.append(mean_squared_error(y_test, preds))
    
    if np.mean(mses) < mejor_mse:
        mejor_mse = np.mean(mses)
        mejor_alpha = a

mejor_alpha, mejor_mse

(0.2612675225563329, 2.9087973868443227)

In [8]:
B = 1000
coef_ridge_boot = []

for b in range(B):
    i = np.random.choice(n, n, replace=True)
    X_b = X_todo.iloc[i]
    y_b = y_todo.iloc[i]

    ridge = Ridge(alpha=mejor_alpha, fit_intercept=False)
    ridge.fit(X_b, y_b)

    coef_ridge_boot.append(ridge.coef_)

coef_ridge_boot = np.array(coef_ridge_boot)

ridge_boot_std = coef_ridge_boot.std(axis=0)
ridge_boot_std

array([0.32975796, 0.00189331, 0.01087069, 0.00637242])

## DEFAULT

In [9]:
df = pd.read_csv("Default.csv")

In [10]:
x_todo = df[['balance', 'income', 'student']]
x_todo = pd.get_dummies(x_todo, columns=['student'], drop_first=True)
x_todo = x_todo.astype(np.float64)
X = sm.add_constant(x_todo)
y = (df["default"] == "Yes").astype(int)

In [11]:
logit = sm.Logit(y, X)
results = logit.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.078577
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:                Thu, 20 Nov 2025   Pseudo R-squ.:                  0.4619
Time:                        17:29:51   Log-Likelihood:                -785.77
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                3.257e-292
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const         -10.8690      0.492    -22.079      0.000     -11.834      -9.904
balance         0.0057      0.000     24.737      0.000       0.005       0.006
income       3.033e-06    8.2e-06      0.370      0.712    -1.3e-05    1.91e-05
student_Yes    -0.6468      0.236     -2.738      0.006      -1.110      -0.184
===============================================================================

Possibly complete quasi-separation: A fraction 0.15 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [12]:
coef_logit = results.params
se_logit = results.bse

In [13]:
B = 1000
coef_boot = []

n = len(df)

for b in range(B):
    i = np.random.choice(n, n, replace=True)
    X_b = X.iloc[i]
    y_b = y.iloc[i]

    modelo_b = sm.Logit(y_b, X_b).fit(disp=False)
    coef_boot.append(modelo_b.params.values)

coef_boot = np.array(coef_boot)

coef_boot_media = coef_boot.mean(axis=0)
coef_boot_std = coef_boot.std(axis=0)

In [14]:
comparacion_default = pd.DataFrame({
    "Coef Logit": coef_logit.values,
    "SE Logit": se_logit.values,
    "Mean Bootstrap": coef_boot_media,
    "Std Bootstrap": coef_boot_std
}, index=X.columns)

comparacion_default

,Coef Logit,SE Logit,Mean Bootstrap,Std Bootstrap
const,-10.869045,0.492273,-10.925291,0.524532
balance,0.005737,0.000232,0.005760,0.000235
income,0.000003,0.000008,0.000003,0.000009
student_Yes,-0.646776,0.236257,-0.638561,0.255041


Como se mencionó previamente, los modelos originales difieren del modelo con bootstrap porque éste suele reflejar resultados más "reales", o que toman en cuenta la variabilidad, en vez de tomar por hecho supuestos de cada modelo. 